In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
"""读入数据"""
data_A = pd.read_csv(r'../input/datasets-ymz/train_0.csv')
data_B = pd.read_csv(r'../input/datasets-ymz/test_0.csv')

In [ ]:
train_data = data_A.iloc[:105000,:]
print(len(train_data))

validation_data = data_A.iloc[105000:,:]
print(len(validation_data))

test_data = data_B.iloc[:,:]
print(len(test_data))

In [ ]:
# train_data = pd.read_csv(r'../input/y-datasets/Train_Merge_Data.csv')
# print(len(train_data))
# train_dev = pd.read_csv(r'../input/y-datasets/train.csv')
# print(len(train_dev))
# validation_data = pd.read_csv(r'../input/y-datasets/Validation_Merge_Data.csv')
# print(len(validation_data))
# validation_dev = pd.read_csv(r'../input/y-datasets/validation.csv')
# print(len(validation_dev))

In [ ]:
"""加载数据集train_data"""
Train_label_Sbp = train_data.iloc[:,1800]
Train_label_Sbp = Train_label_Sbp.values

Train_label_Dbp = train_data.iloc[:,1801]
Train_label_Dbp = Train_label_Dbp.values

Train_Dev_One = train_data.iloc[:,600:1200]
Train_Dev_One = Train_Dev_One.values

Train_Dev_Two = train_data.iloc[:,1200:1800]
Train_Dev_Two = Train_Dev_Two.values

Train_Data = train_data.iloc[:,:600]
Train_Data = Train_Data.values

In [ ]:
# """加载数据集train_data"""
# Train_label_Sbp = train_data.iloc[:,650]
# Train_label_Sbp = Train_label_Sbp.values

# Train_label_Dbp = train_data.iloc[:,651]
# Train_label_Dbp = Train_label_Dbp.values

# Train_Dev_One = train_dev.iloc[:,:640]
# Train_Dev_One = Train_Dev_One.values

# Train_Dev_Two = train_dev.iloc[:,640:]
# Train_Dev_Two = Train_Dev_Two.values

# Train_Data = train_data.iloc[:,:640]
# Train_Data = Train_Data.values

In [ ]:
# """加载数据集validation_data"""
# Validation_label_Sbp = validation_data.iloc[:,650]
# Validation_label_Sbp = Validation_label_Sbp.values

# Validation_label_Dbp = validation_data.iloc[:,651]
# Validation_label_Dbp = Validation_label_Dbp.values

# Validation_Dev_One = validation_dev.iloc[:,:640]
# Validation_Dev_One = Validation_Dev_One.values

# Validation_Dev_Two = validation_dev.iloc[:,640:]
# Validation_Dev_Two = Validation_Dev_Two.values

# Validation_Data = validation_data.iloc[:,:640]
# Validation_Data = Validation_Data.values

In [ ]:
"""加载数据集validation_data"""
Validation_label_Sbp = validation_data.iloc[:,1800]
Validation_label_Sbp = Validation_label_Sbp.values

Validation_label_Dbp = validation_data.iloc[:,1801]
Validation_label_Dbp = Validation_label_Dbp.values

Validation_Dev_One = validation_data.iloc[:,600:1200]
Validation_Dev_One = Validation_Dev_One.values

Validation_Dev_Two = validation_data.iloc[:,1200:1800]
Validation_Dev_Two = Validation_Dev_Two.values

Validation_Data = validation_data.iloc[:,:600]
Validation_Data = Validation_Data.values

In [ ]:
"""数据格式、尺寸"""
print("data_information:")
print(Train_Data.shape)
print(Train_Dev_One.shape)
print(Train_Dev_Two.shape)
print(Train_label_Sbp.shape)
print(Train_label_Dbp.shape)

print("data_information:")
print(Validation_Data.shape)
print(Validation_Dev_One.shape)
print(Validation_Dev_Two.shape)
print(Validation_label_Sbp.shape)
print(Validation_label_Dbp.shape)

In [ ]:
"""注意力机制模块"""
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
# 注意力机制：Relu6方式
# 激活函数 relu6
def relu6(x):
    return tf.keras.activations.relu(x, max_value=6)
#   利用relu函数乘上x模拟sigmoid
def hard_swish(x):
    return x * tf.keras.activations.relu(x + 3.0, max_value=6.0) / 6.0

#---------------------------------------#
#   通道注意力机制单元
#   利用两次全连接算出每个通道的比重
#   可以连接在任意特征层后面
#---------------------------------------#

def squeeze(inputs):
    input_channels = int(inputs.shape[-1])
    
    x = layers.GlobalAveragePooling1D()(inputs)

    x = layers.Dense(int(input_channels/4))(x)
    x = relu6(x)

    x = layers.Dense(input_channels)(x)
    x = hard_swish(x)

    x = layers.Reshape((1, input_channels))(x)
    #print(x)
    #print(inputs)
    x = layers.Multiply()([inputs, x])
    return x


In [ ]:
"""下采样"""
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
#基础层—3x3_3x3—下采样——改变卷积层步长的方式
#ps:目前试验结果中最好的下采样方式
def DownSampling(inputs,filters,stride_padding='same',Drop=False):
    layer = layers.Conv1D(filters,3,padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    
    res_layer = layers.Conv1D(filters,1,padding='same')(inputs)
    res_layer = layers.BatchNormalization()(res_layer)
    layer = layers.add([layer,res_layer])
    
    layer = layers.Activation(tf.nn.relu)(layer)
    
    if Drop==True:
        layer = layers.Dropout(0.25)(layer)
    
    pool_inputs = layers.Conv1D(filters,3,padding=stride_padding,strides=2)(layer)
    pool = layers.BatchNormalization()(pool_inputs)
    pool = layers.Activation(tf.nn.relu)(pool)

    pool = layers.Conv1D(filters,3,padding='same')(pool)
    pool = layers.BatchNormalization()(pool)
    
    res_pool = layers.Conv1D(filters,1,padding='same')(pool_inputs)
    res_pool = layers.BatchNormalization()(res_pool)
    pool = layers.add([pool,res_pool])
    
    pool = layers.Activation(tf.nn.relu)(pool)
    
    
    return layer,pool


In [ ]:
"""上采样"""
#基础层上采样
def UpSampling(inputs,con_input,filters,need_zero=False):
    
    
    up_layer = layers.UpSampling1D(size=2)(inputs)
    
    if need_zero==True:
        up_layer = layers.ZeroPadding1D((0,1))(up_layer)
    
    layer = layers.Conv1D(filters,2,padding='same')(up_layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.Concatenate(axis=2)([layer,con_input])
    
    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    output = layers.Activation(tf.nn.relu)(layer)
    
    return output

In [ ]:
"""上采样结束的卷积操作"""
def FinalConv1D(inputs,filters):
    layer = layers.Conv1D(filters,3,padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.MaxPooling1D(pool_size=2)(layer)
    
    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.MaxPooling1D(pool_size=2)(layer)
    return layer

In [ ]:
"""上采样结束的卷积操作"""
def Inception_model(inputs,filters,kernel_size=3,stride_padding='same',Drop=False):
    layer = layers.Conv1D(filters,kernel_size=kernel_size,padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,kernel_size=kernel_size,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    
    res_layer = layers.Conv1D(filters,1,padding='same')(inputs)
    res_layer = layers.BatchNormalization()(res_layer)
    layer = layers.add([layer,res_layer])
    
    layer = layers.Activation(tf.nn.relu)(layer)
    
    if Drop==True:
        layer = layers.Dropout(0.25)(layer)
    
    pool_inputs = layers.Conv1D(filters,kernel_size=kernel_size,padding=stride_padding,strides=2)(layer)
    pool = layers.BatchNormalization()(pool_inputs)
    pool = layers.Activation(tf.nn.relu)(pool)

    pool = layers.Conv1D(filters,kernel_size=kernel_size,padding='same')(pool)
    pool = layers.BatchNormalization()(pool)
    
    res_pool = layers.Conv1D(filters,1,padding='same')(pool_inputs)
    res_pool = layers.BatchNormalization()(res_pool)
    pool = layers.add([pool,res_pool])
    
    pool = layers.Activation(tf.nn.relu)(pool)
    return layer,pool

In [ ]:
#ARU-Net原版
def create_model():
    #双输入模型
    inputs_1 = keras.Input(shape=(600,1),name='inputs_1')
    inputs_dev1 = keras.Input(shape=(600,1),name='inputs_dev1')
    inputs_dev2 = keras.Input(shape=(600,1),name='inputs_dev2')
    
    inputs = layers.Concatenate(axis=2)([inputs_1,inputs_dev1,inputs_dev2])
    
    #下采样
    layer1,pool1 = DownSampling(inputs,filters=64)
    
    layer2,pool2 = DownSampling(pool1,filters=128)
        
    layer3,pool3 = DownSampling(pool2,filters=256)
    
    layer4,pool4 = DownSampling(pool3,filters=512,stride_padding='valid',Drop=False)
    
    #最后一层不需要Pooling
#     layer5,pool5 = DownSampling(pool4,filters=1024,Drop=False)

    layer51,pool51 = Inception_model(pool4,filters=1024,kernel_size=1)
    layer52,pool52 = Inception_model(pool4,filters=1024,kernel_size=3)
    layer53,pool53 = Inception_model(pool4,filters=1024,kernel_size=5)
    
    layer51 = squeeze(layer51)
    layer52 = squeeze(layer52)
    layer53 = squeeze(layer53)
    
    layer51 = layers.BatchNormalization()(layer51)
    layer52 = layers.BatchNormalization()(layer52)
    layer53 = layers.BatchNormalization()(layer53)
    layer5 = layers.add([layer51,layer52])
    layer5 = layers.add([layer5,layer53])
    #上采样
    
    #attention
    layer4 = squeeze(layer4)
    
    layer44 = UpSampling(layer5,layer4,512,need_zero=True)
    
    #attention
    layer3 = squeeze(layer3)
                     
    layer33 = UpSampling(layer44,layer3,256)
    
    #attention
    layer2 = squeeze(layer2)
    
    layer22 = UpSampling(layer33,layer2,128)
    
    #attention
    layer1 = squeeze(layer1)
    
    layer11 = UpSampling(layer22,layer1,64)
    
    
    layer = layers.GlobalAveragePooling1D()(layer11)

    outputs_sbp = layers.Dense(1,name='Sbp')(layer)
    outputs_dbp = layers.Dense(1,name='Dbp')(layer)

    model = keras.Model(inputs=[inputs_1,inputs_dev1,inputs_dev2],outputs=[outputs_sbp,outputs_dbp])
    #model = keras.Model(inputs=[inputs_1],outputs=[outputs_sbp,outputs_dbp])
    return model

In [ ]:
"""自定义评价指标模块"""
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(abs(y_pred-y_true))
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square(abs(y_pred-y_true) - u)))

In [ ]:
"""回调函数"""
#保存迭代周期内最好的模型
checkpoint_filepath = r'./model_struction.h5'
Save_epochs = 100 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_mean_absolute_error',
    mode='min',
    save_best_only=True)

In [ ]:
"""建立模型"""
with strategy.scope():
    model = create_model()
    
    model.compile(loss={'Sbp':"mse",'Dbp':"mse"}, optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])

In [ ]:
model.summary()

In [ ]:
"""保存模型结构图片"""
tf.keras.utils.plot_model(model, to_file=r'./model_graph.png', show_shapes=True, show_layer_names=False)


In [ ]:
history = model.fit({'inputs_1':Train_Data,'inputs_dev1':Train_Dev_One,'inputs_dev2':Train_Dev_Two},{'Sbp':Train_label_Sbp,'Dbp':Train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=({'inputs_1':Validation_Data,'inputs_dev1':Validation_Dev_One,'inputs_dev2':Validation_Dev_Two},{'Sbp':Validation_label_Sbp,'Dbp':Validation_label_Dbp})
                    )

In [ ]:
# history = model.fit({'inputs_1':Train_Data},{'Sbp':Train_label_Sbp,'Dbp':Train_label_Dbp},
#                     batch_size=128*8,
#                     epochs=500,
#                     callbacks=model_checkpoint_callback,
#                     validation_data=({'inputs_1':Validation_Data},{'Sbp':Validation_label_Sbp,'Dbp':Validation_label_Dbp})
#                     )

In [ ]:
with open(r'./last_model.h5.pickle', 'wb') as file_pi:
 	pickle.dump(history.history, file_pi)

In [ ]:
# x=32
# def create_model():
    
#     inputs_1 = keras.Input(shape=(600,1),name='inputs_1')
#     inputs_dev1 = keras.Input(shape=(600,1),name='inputs_dev1')
#     inputs_dev2 = keras.Input(shape=(600,1),name='inputs_dev2')
    
#     inputs = layers.Concatenate(axis=1)([inputs_1,inputs_dev1,inputs_dev2])
    
#     inputs = keras.Input(shape=(1800,1))

#     conv1 = layers.Conv1D(x*2,3,padding='same')(inputs)
#     conv1 = layers.BatchNormalization()(conv1)
#     conv1 = layers.Activation(tf.nn.relu)(conv1)

#     conv1 = layers.Conv1D(x*2,3,padding='same')(conv1)
#     conv1 = layers.BatchNormalization()(conv1)
#     conv1 = layers.Activation(tf.nn.relu)(conv1)

#     pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
#     conv2 = layers.Conv1D(x*4,3,padding='same')(pool1)
#     conv2 = layers.BatchNormalization()(conv2)
#     conv2 = layers.Activation(tf.nn.relu)(conv2)

#     conv2 = layers.Conv1D(x*4,3,padding='same')(conv2)
#     conv2 = layers.BatchNormalization()(conv2)
#     conv2 = layers.Activation(tf.nn.relu)(conv2)

#     pool2 = layers.MaxPooling1D(pool_size=2)(conv2)
    
    
#     conv3 = layers.Conv1D(x*8,3,padding='same')(pool2)
#     conv3 = layers.BatchNormalization()(conv3)
#     conv3 = layers.Activation(tf.nn.relu)(conv3)

#     conv3 = layers.Conv1D(x*8,3,padding='same')(conv3)
#     conv3 = layers.BatchNormalization()(conv3)
#     conv3 = layers.Activation(tf.nn.relu)(conv3)

#     pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    
#     conv4 = layers.Conv1D(x*16,3,padding='same')(pool3)
#     conv4 = layers.BatchNormalization()(conv4)
#     conv4 = layers.Activation(tf.nn.relu)(conv4)

#     conv4 = layers.Conv1D(x*16,3,padding='same')(conv4)
#     conv4 = layers.BatchNormalization()(conv4)
#     conv4 = layers.Activation(tf.nn.relu)(conv4)
    
#     conv4 = layers.Dropout(0.5)(conv4)
#     ####
#     pool4 = layers.MaxPooling1D(pool_size=2)(conv4)
    

    
#     conv5 = layers.Conv1D(x*32,3,padding='same')(pool4)
#     conv5 = layers.BatchNormalization()(conv5)
#     conv5 = layers.Activation(tf.nn.relu)(conv5)

#     conv5 = layers.Conv1D(x*32,3,padding='same')(conv5)
#     conv5 = layers.BatchNormalization()(conv5)
#     conv5 = layers.Activation(tf.nn.relu)(conv5)
    
#     conv5 = layers.Dropout(0.5)(conv5)
    
    
#     ####
#     conv5 = layers.UpSampling1D(size=2)(conv5)
#     conv5 = layers.ZeroPadding1D((0,1))(conv5)
    
#     conv5 = layers.Conv1D(x*16,2,padding='same')(conv5)
    
#     up6 = layers.Concatenate(axis=2)([conv5,conv4])
    
#     conv6 = layers.Conv1D(x*16,3,padding='same')(up6)
#     conv6 = layers.BatchNormalization()(conv6)
#     conv6 = layers.Activation(tf.nn.relu)(conv6)

#     conv6 = layers.Conv1D(x*16,3,padding='same')(conv6)
#     conv6 = layers.BatchNormalization()(conv6)
#     conv6 = layers.Activation(tf.nn.relu)(conv6)
    
#     ####
#     conv6 = layers.UpSampling1D(size=2)(conv6)
    
#     conv6 = layers.Conv1D(x*8,2,padding='same')(conv6)
#     conv6 = layers.BatchNormalization()(conv6)
#     conv6 = layers.Activation(tf.nn.relu)(conv6)
    
#     up7 = layers.Concatenate(axis=2)([conv6,conv3])

#     conv7 = layers.Conv1D(x*8,3,padding='same')(up7)
#     conv7 = layers.BatchNormalization()(conv7)
#     conv7 = layers.Activation(tf.nn.relu)(conv7)

#     conv7 = layers.Conv1D(x*8,3,padding='same')(conv7)
#     conv7 = layers.BatchNormalization()(conv7)
#     conv7 = layers.Activation(tf.nn.relu)(conv7)
    
#     ####
    
#     conv7 = layers.UpSampling1D(size=2)(conv7)
    
#     conv7 = layers.Conv1D(x*4,2,padding='same')(conv7)
#     conv7 = layers.BatchNormalization()(conv7)
#     conv7 = layers.Activation(tf.nn.relu)(conv7)
    
#     up8 = layers.Concatenate(axis=2)([conv7,conv2])

#     conv8 = layers.Conv1D(x*4,3,padding='same')(up8)
#     conv8 = layers.BatchNormalization()(conv8)
#     conv8 = layers.Activation(tf.nn.relu)(conv8)

#     conv8 = layers.Conv1D(x*4,3,padding='same')(conv8)
#     conv8 = layers.BatchNormalization()(conv8)
#     conv8 = layers.Activation(tf.nn.relu)(conv8)
    
#     ####
    
#     conv8 = layers.UpSampling1D(size=2)(conv8)
    
#     conv8 = layers.Conv1D(x*2,2,padding='same')(conv8)
#     conv8 = layers.BatchNormalization()(conv8)
#     conv8 = layers.Activation(tf.nn.relu)(conv8)
    

#     up9 = layers.Concatenate(axis=2)([conv8,conv1])

#     conv9 = layers.Conv1D(x*2,3,padding='same')(up9)
#     conv9 = layers.BatchNormalization()(conv9)
#     conv9 = layers.Activation(tf.nn.relu)(conv9)

#     conv9 = layers.Conv1D(x*2,3,padding='same')(conv9)
#     conv9 = layers.BatchNormalization()(conv9)
#     conv9 = layers.Activation(tf.nn.relu)(conv9)

#     # 论文方法
#     conv10 = layers.Conv1D(2,1,padding='same')(conv9)
#     conv10 = layers.BatchNormalization()(conv10)
#     conv10 = layers.Activation(tf.nn.relu)(conv10)
    
#     conv11 = layers.Conv1D(1,1,padding='same')(conv10)
#     conv11 = layers.BatchNormalization()(conv11)
#     conv11 = layers.Activation(tf.nn.relu)(conv11)
    
#     com_layer = layers.Flatten()(conv11)
#     #com_layer = layers.GlobalAveragePooling1D()(conv10)
    
#     #com_layer = layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.4))(com_layer)
    
#     outputs_sbp = layers.Dense(1,name='Sbp')(com_layer)
#     outputs_dbp = layers.Dense(1,name='Dbp')(com_layer)

#     model = keras.Model(inputs=[inputs_1,inputs_dev1,inputs_dev2],outputs=[outputs_sbp,outputs_dbp])
    
#     return model

In [ ]:
"""回调函数"""
#保存迭代周期内最好的模型
checkpoint_filepath = r'./model_struction_1.h5'
Save_epochs = 100 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_mean_absolute_error',
    mode='min',
    save_best_only=True)

In [ ]:
"""建立模型"""
with strategy.scope():
    model = create_model()
    
    model.compile(loss={'Sbp':"mse",'Dbp':"mse"}, optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])

In [ ]:
model.summary()

In [ ]:
"""保存模型结构图片"""
tf.keras.utils.plot_model(model, to_file=r'./model_graph_1.png', show_shapes=True, show_layer_names=False)


In [ ]:
history = model.fit({'inputs_1':Train_Data,'inputs_dev1':Train_Dev_One,'inputs_dev2':Train_Dev_Two},{'Sbp':Train_label_Sbp,'Dbp':Train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=({'inputs_1':Validation_Data,'inputs_dev1':Validation_Dev_One,'inputs_dev2':Validation_Dev_Two},{'Sbp':Validation_label_Sbp,'Dbp':Validation_label_Dbp})
                    )

In [ ]:
with open(r'./last_model_1.h5.pickle', 'wb') as file_pi:
 	pickle.dump(history.history, file_pi)